# Instalação e importação das bibliotecas

In [2]:
!pip install numpy pandas matplotlib scikit-learn opencv-python tensorflow keras opencv-python-headless keras-tuner

In [3]:
import os
import cv2
import shutil
import random
import numpy as np
import tensorflow as tf
import keras_tuner as kt
from sklearn.model_selection import KFold
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, regularizers, optimizers, callbacks

2024-10-14 23:39:07.240475: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-14 23:39:07.504026: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-14 23:39:07.787098: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-14 23:39:08.010054: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-14 23:39:08.065253: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-14 23:39:08.406445: I tensorflow/core/platform/cpu_feature_gu

**Atenção:** Caso sua máquina não tenha GPU NVIDIA, aparecerá um aviso acima informando que não foram encontrados ***drivers CUDA*** (Compute Unified Device Architeture) que são uma parte essencial do ecossistema de programação paralela da NVIDIA, que permite o uso de GPUs NVIDIA para realizar cálculos intensivos. Porém, já que os drivers não foram encontrados será usada a CPU da máquina ao invés da GPU, o que **não** irá interferir na acurácia do modelo, apenas no tempo de treinamento.

# Coleta e processamento de dados

## Pré-processamento dos dados

In [4]:
def preprocess_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img_resized = cv2.resize(img, (224, 224))
    img_normalized = img_resized / 255.0
    return img_normalized

## Balanceamento do DataSet

Para evitar maior peso em um determinado tipo de dado, foi feito um balanceamento do dataset.

Para isso, foi utilizado o método de reamostragem chamado de undersampling, no qual foi selecionado aleatoriamente os dados de uma classe e duplicado até que a quantidade de dados de cada classe se iguale.

In [5]:
dataset_dir = './dataset'
balanced_dataset_dir = './balanced-dataset'

classes = ['healthy', 'kyphosis', 'lordosis']

os.makedirs(balanced_dataset_dir, exist_ok=True)

image_counts = {}
for cls in classes:
    cls_path = os.path.join(dataset_dir, cls)
    image_counts[cls] = len(os.listdir(cls_path))

max_images = max(image_counts.values())
new_image_counts = {}

for cls in classes:
    os.makedirs(os.path.join(balanced_dataset_dir, cls), exist_ok=True)
    
    images = os.listdir(os.path.join(dataset_dir, cls))
    
    for index, image in enumerate(images):
        new_name = f"{cls}_{index + 1:03d}.jpg"
        shutil.copy(os.path.join(dataset_dir, cls, image), 
                    os.path.join(balanced_dataset_dir, cls, new_name))
    
    new_image_counts[cls] = len(images)

    current_count = new_image_counts[cls]
    while current_count < max_images:
        image_to_copy = random.choice(images)
        
        new_name = f"{cls}_{current_count + 1:03d}.jpg"
        shutil.copy(os.path.join(dataset_dir, cls, image_to_copy),
                    os.path.join(balanced_dataset_dir, cls, new_name))
        
        current_count += 1

print("Dataset balanceado criado em:", balanced_dataset_dir)

Dataset balanceado criado em: ./balanced-dataset


# Criando CNN (rede neural convolucional)

Essa função cria um modelo CNN que passa por algumas camadas convolucionais para extrair características das imagens, nas quais foram utilizados as funções:

- ReLU
    - Função de ativação que permite que a rede neural aprenda padrões complexos
- Pooling
    - Operação de amostragem usada em redes convolucionais para reduzir as dimensões e/ou tamanho da imagem ou das saídas das camadas convolucionais, porém, preservando suas características mais importantes
- Normalização de Batch
    - Normalização de dados de entrada para que a rede neural possa aprender mais rapidamente
- Softmax
    - Função de ativação que converte um vetor de valores reais em uma distribuição de probabilidade

In [6]:
def create_3_class_cnn(input_shape):
    inputs = layers.Input(shape=input_shape)
    
    x = layers.Conv2D(3, (1, 1), padding="same")(inputs)
    
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    
    x = base_model(x)
    
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)

    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(3, activation='softmax')(x)
    
    model = models.Model(inputs=inputs, outputs=outputs)
    
    return model

def create_2_class_cnn(input_shape):
    inputs = layers.Input(shape=input_shape)
    
    x = layers.Conv2D(3, (1, 1), padding="same")(inputs)
    
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    
    x = base_model(x)
    
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)

    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(2, activation='softmax')(x)
    
    model = models.Model(inputs=inputs, outputs=outputs)
    
    return model

input_shape = (224, 224, 1)

generalist_cnn_model = create_3_class_cnn(input_shape)
lordosis_cnn_model = create_2_class_cnn(input_shape)
kiphosis_cnn_model = create_2_class_cnn(input_shape)

lr_scheduler = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001)

# Gerador de alterações nas imagens

Para melhorar a amplitude do treinamento, foi utilizado o ImageDataGenerator, que trás alterações em rotação, deslocamento, zoom, brilho e direção.

In [7]:
train_datagen = ImageDataGenerator(
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    brightness_range = [0.8, 1.2],
    validation_split = 0.3
)

# Pré-processamento dos modelos

In [8]:
generalist_classes = ['healthy', 'kyphosis', 'lordosis']

generalist_train_generator = train_datagen.flow_from_directory(
    './balanced-dataset',
    target_size = (224, 224),
    color_mode = 'grayscale',
    batch_size = 4,
    class_mode = 'categorical',
    subset = 'training'
)

generalist_validation_generator = train_datagen.flow_from_directory(
    './balanced-dataset',
    target_size = (224, 224),
    color_mode = 'grayscale',
    batch_size = 4,
    class_mode = 'categorical',
    subset = 'validation'
)

generalist_cnn_model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

Found 36 images belonging to 3 classes.
Found 15 images belonging to 3 classes.


In [9]:
lordosis_classes = ['healthy', 'lordosis']

lordosis_train_generator = train_datagen.flow_from_directory(
    './balanced-dataset',
    target_size = (224, 224),
    color_mode = 'grayscale',
    batch_size = 4,
    class_mode = 'categorical',
    subset = 'training',
    classes = lordosis_classes
)

lordosis_validation_generator = train_datagen.flow_from_directory(
    './balanced-dataset',
    target_size = (224, 224),
    color_mode = 'grayscale',
    batch_size = 4,
    class_mode = 'categorical',
    subset = 'validation',
    classes = lordosis_classes
)

lordosis_cnn_model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

Found 24 images belonging to 2 classes.
Found 10 images belonging to 2 classes.


In [10]:
kiphosis_classes = ['healthy', 'kyphosis']

kiphosis_train_generator = train_datagen.flow_from_directory(
    './balanced-dataset',
    target_size = (224, 224),
    color_mode = 'grayscale',
    batch_size = 4,
    class_mode = 'categorical',
    subset = 'training',
    classes = kiphosis_classes
)

kiphosis_validation_generator = train_datagen.flow_from_directory(
    './balanced-dataset',
    target_size = (224, 224),
    color_mode = 'grayscale',
    batch_size = 4,
    class_mode = 'categorical',
    subset = 'validation',
    classes = kiphosis_classes
)

kiphosis_cnn_model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

Found 24 images belonging to 2 classes.
Found 10 images belonging to 2 classes.


# Treinamento do modelo

In [11]:
generalist_history = generalist_cnn_model.fit(
    generalist_train_generator,
    validation_data = generalist_validation_generator,
    epochs = 30,
    callbacks = [lr_scheduler]
)

lordosis_history = lordosis_cnn_model.fit(
    lordosis_train_generator,
    validation_data = lordosis_validation_generator,
    epochs = 30,
    callbacks = [lr_scheduler]
)

kiphosis_history = kiphosis_cnn_model.fit(
    kiphosis_train_generator,
    validation_data = kiphosis_validation_generator,
    epochs = 30,
    callbacks = [lr_scheduler]
)

/opt/conda/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 56s 4s/step - accuracy: 0.3358 - loss: 4.4626 - val_accuracy: 0.3333 - val_loss: 4.3427 - learning_rate: 0.0010
Epoch 2/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 32s 4s/step - accuracy: 0.4501 - loss: 4.2209 - val_accuracy: 0.4000 - val_loss: 3.8421 - learning_rate: 0.0010
Epoch 3/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 30s 3s/step - accuracy: 0.3832 - loss: 3.9825 - val_accuracy: 0.3333 - val_loss: 3.6575 - learning_rate: 0.0010
Epoch 4/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 25s 3s/step - accuracy: 0.7386 - loss: 3.3767 - val_accuracy: 0.6000 - val_loss: 3.5524 - learning_rate: 0.0010
Epoch 5/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 27s 3s/step - accuracy: 0.6695 - loss: 3.2548 - val_accuracy: 0.4667 - val_loss: 3.5182 - learning_rate: 0.0010
Epoch 6/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 45s 3s/step - accuracy: 0.6395 - loss: 3.3289 - val_accuracy: 0.4000 - val_loss: 3.6569 - learning_rate: 0.0010
Epoch 7/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 33s 4s/step - accuracy: 0.4002 - loss: 3.8582 - val_accuracy: 0.4667 - val_l

### Porque resultados sempre são diferentes:

- Ao inicializar o modelo os pesos da rede neural são inicializados de forma aleatória, e como o processo de otimização do modelo começa a partir de diferentes pontos, cada execução pode levar a resultados diferentes, por isso, cada vez que o código for executado os resultados não serão exatamente os mesmos, mas aproximados.
- 
Image Augmentation: técnicas de rotação, deslocamento, zoom e etc. aplicadas na imagem para criar novas variações para realizar o treinamento do modelo, sendo também um processo aleatório.
- 
Shuffling: Durante o treinamento, os dados de treino são embaralhados a cada época. Isso garante que o modelo não aprenda de forma dependente da ordem dos exemplos, mas também pode fazer com que os resultados variem.io.

# Avaliação dos Modelos

In [12]:
generalist_val_loss, generalist_val_acc = generalist_cnn_model.evaluate(generalist_validation_generator)
lordosis_val_loss, lordosis_val_acc = lordosis_cnn_model.evaluate(lordosis_validation_generator)
kiphosis_val_loss, kiphosis_val_acc = kiphosis_cnn_model.evaluate(kiphosis_validation_generator)

print(f"Validação modelo generalista - Loss: {generalist_val_loss}, Acurácia: {generalist_val_acc}")
print(f"Validação modelo de lordose - Loss: {lordosis_val_loss}, Acurácia: {lordosis_val_acc}")
print(f"Validação modelo de cifose - Loss: {kiphosis_val_loss}, Acurácia: {kiphosis_val_acc}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.7583 - loss: 2.6461
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 761ms/step - accuracy: 0.8062 - loss: 2.4203
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 845ms/step - accuracy: 0.9187 - loss: 2.1676
Validação modelo generalista - Loss: 2.7754969596862793, Acurácia: 0.6666666865348816
Validação modelo de lordose - Loss: 2.4447550773620605, Acurácia: 0.800000011920929
Validação modelo de cifose - Loss: 2.192209482192993, Acurácia: 0.8999999761581421


# Validação Cruzada dos modelos e ajustes do Hiperparâmetros

#### Contruir o modelo com hiperparâmetros variáveis

In [13]:
def build_model(hp, num_classes):
    inputs = layers.Input(shape=(224, 224, 1))
    
    x = layers.Conv2D(3, (1, 1), padding="same")(inputs)
    
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    x = base_model(x)
    
    x = layers.Conv2D(hp.Int('conv_units', min_value=32, max_value=128, step=32), 
                      (3, 3), activation='relu', padding='same',
                      kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    
    x = layers.Flatten()(x)
    x = layers.Dense(hp.Int('dense_units', min_value=64, max_value=256, step=64), 
                     activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Dropout(hp.Float('dropout', min_value=0.3, max_value=0.7, step=0.1))(x)
    
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = models.Model(inputs=inputs, outputs=outputs)
    
    model.compile(
        optimizer=hp.Choice('optimizer', ['adam', 'rmsprop']),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

#### Parâmetros de Configuração

In [14]:
batch_size = 4
input_shape = (224, 224, 1)
dataset_path = './balanced-dataset'

#### Ajuste dos hiperparâmetros e validação cruzada

In [15]:
def cross_val_and_tuning(classes, num_classes, epochs=10, n_splits=3):
    all_data_generator = train_datagen.flow_from_directory(
        dataset_path,
        target_size=(224, 224),
        color_mode='grayscale',
        batch_size=batch_size,
        class_mode='categorical',
        subset='training',
        classes=classes
    )

    n_samples = all_data_generator.samples 
    if n_splits > n_samples:
        raise ValueError(f"Número de splits {n_splits} é maior que o número de amostras {n_samples}.")

    def model_builder(hp):
        return build_model(hp, num_classes)

    tuner = kt.Hyperband(
        model_builder,
        objective='val_accuracy',
        max_epochs=epochs,
        factor=3,
        directory='tuning_results',
        project_name='hyperparameter_tuning'
    )

    kfold = KFold(n_splits=n_splits, shuffle=True)
    split = 1
    for train_index, val_index in kfold.split(range(n_samples)):
        print(f'Fold {split} de {n_splits}')
        
        train_generator = train_datagen.flow_from_directory(
            dataset_path,
            target_size=(224, 224),
            color_mode='grayscale',
            batch_size=batch_size,
            class_mode='categorical',
            subset='training',
            classes=classes
        )
        
        validation_generator = train_datagen.flow_from_directory(
            dataset_path,
            target_size=(224, 224),
            color_mode='grayscale',
            batch_size=batch_size,
            class_mode='categorical',
            subset='validation',
            classes=classes
        )

        tuner.search(train_generator, validation_data=validation_generator, epochs=epochs)

        best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
        print(f'Melhores hiperparâmetros no Fold {split}:')
        print(f'Conv2D units: {best_hps.get("conv_units")}')
        print(f'Dense units: {best_hps.get("dense_units")}')
        print(f'Dropout: {best_hps.get("dropout")}')
        print(f'Otimizador: {best_hps.get("optimizer")}')

        split += 1

#### Ajustes para os modelos

In [16]:
generalist_classes = ['healthy', 'kyphosis', 'lordosis']
cross_val_and_tuning(generalist_classes, num_classes=3)

lordosis_classes = ['healthy', 'lordosis']
cross_val_and_tuning(lordosis_classes, num_classes=2)

kiphosis_classes = ['healthy', 'kyphosis']
cross_val_and_tuning(kiphosis_classes, num_classes=2)

Trial 30 Complete [00h 03m 13s]
val_accuracy: 0.7333333492279053

Best val_accuracy So Far: 0.9333333373069763
Total elapsed time: 02h 28m 13s
Melhores hiperparâmetros no Fold 1:
Conv2D units: 32
Dense units: 64
Dropout: 0.3
Otimizador: rmsprop
Fold 2 de 3
Found 36 images belonging to 3 classes.
Found 15 images belonging to 3 classes.
Melhores hiperparâmetros no Fold 2:
Conv2D units: 32
Dense units: 64
Dropout: 0.3
Otimizador: rmsprop
Fold 3 de 3
Found 36 images belonging to 3 classes.
Found 15 images belonging to 3 classes.
Melhores hiperparâmetros no Fold 3:
Conv2D units: 32
Dense units: 64
Dropout: 0.3
Otimizador: rmsprop
Found 24 images belonging to 2 classes.
Reloading Tuner from tuning_results/hyperparameter_tuning/tuner0.json
Fold 1 de 3
Found 24 images belonging to 2 classes.
Found 10 images belonging to 2 classes.
Melhores hiperparâmetros no Fold 1:
Conv2D units: 32
Dense units: 64
Dropout: 0.3
Otimizador: rmsprop
Fold 2 de 3
Found 24 images belonging to 2 classes.
Found 10 im

# Função de predição

In [19]:
def predict_image(image_path, model):
    img = preprocess_image(image_path)
    img = np.expand_dims(img, axis=0)
    prediction = model.predict(img)
    predicted_class = np.argmax(prediction)
    return predicted_class, prediction

generalist_predicted_class, generalist_result = predict_image('./test-data/001.jpg', generalist_cnn_model)
lordosis_predicted_class, lordosis_result = predict_image('./test-data/001.jpg', lordosis_cnn_model)
kiphosis_predicted_class, kiphosis_result = predict_image('./test-data/001.jpg', kiphosis_cnn_model)

print(f'Predição (modelo generalista): {generalist_result}')
print(f'Classe prevista (modelo generalista): {generalist_classes[generalist_predicted_class]}')

print(f'Predição (modelo de lordose): {lordosis_result}')
print(f'Classe prevista (modelo de lordose): {lordosis_classes[lordosis_predicted_class]}')

print(f'Predição (modelo de cifose): {kiphosis_result}')
print(f'Classe prevista (modelo de cifose): {kiphosis_classes[kiphosis_predicted_class]}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
Predição (modelo generalista): [[0.7788245  0.02539187 0.19578362]]
Classe prevista (modelo generalista): healthy
Predição (modelo de lordose): [[0.85335857 0.14664136]]
Classe prevista (modelo de lordose): healthy
Predição (modelo de cifose): [[0.70608634 0.2939137 ]]
Classe prevista (modelo de cifose): healthy
